In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, GRU, Activation
from keras import optimizers 
from keras.activations import tanh
from keras.utils import get_custom_objects

seed = 1234
np.random.seed(seed)
plt.style.use('ggplot')
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
dataraw = pd.read_csv('./BTC-USD.csv',index_col='Date', parse_dates=['Date'])
dataraw = pd.DataFrame(dataraw, )

In [ ]:
dataraw = pd.read_csv('./Data/btcusd_5-min_data.csv')
dataraw['datetime'] = pd.to_datetime(dataraw['Timestamp'], unit='s')
dataraw.set_index('datetime', inplace=True)

In [ ]:
# use feature 'Date' & 'Close'
dataset = pd.DataFrame(dataraw['Close'])
print(' Count row of data: ',len(dataset))

fig = plt.figure(figsize=(14, 6))
plt.plot(dataset)
plt.xlabel('Date')
plt.ylabel('Bitcoin Price')
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m"))
plt.title('Bitcoin Price')
plt.show()

In [ ]:
#Min-Max Normalization
dataset_norm = dataset.copy()
dataset[['Close']]
scaler = MinMaxScaler()
dataset_norm['Close'] = scaler.fit_transform(dataset[['Close']])
dataset_norm

In [ ]:
fig = plt.figure(figsize=(10, 4))
plt.plot(dataset_norm)
plt.xlabel('Date')
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m"))
plt.title('Data Normalized')
plt.show()

In [ ]:
# Partition data into data train, val & test
totaldata = dataset.values
totaldatatest = int(len(totaldata)*0.1)
totaldataval = int(len(totaldata)*0.1)
totaldatatrain = int(len(totaldata)*0.8)

# Store data into each partition
test_set = dataset_norm[0:totaldatatest]
val_set=dataset_norm[totaldatatest:totaldatatest+totaldataval]
training_set = dataset_norm[totaldatatest+totaldataval:]

In [ ]:
# graph of data training
fig = plt.figure(figsize=(10, 4))
plt.plot(training_set)
plt.xlabel('Date')
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m"))
plt.title('Data Training')
plt.show()

In [ ]:
# Initiaton value of lag
lag = 289
# sliding windows function
def create_sliding_windows(data,len_data,lag):
    x=[]
    y=[]
    for i in range(lag, len_data - lag):
        x.append(data[i - lag: i, 0])
        y.append(data[i : i + lag,0]) 
        
    return np.array(x),np.array(y)

# Formating data into array for create sliding windows
array_training_set = np.array(training_set)
array_val_set = np.array(val_set)
array_test_set = np.array(test_set)

# Create sliding windows into training data
x_train, y_train = create_sliding_windows(array_training_set,len(array_training_set), lag)
x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))
# Create sliding windows into validation data
x_val,y_val = create_sliding_windows(array_val_set,len(array_val_set),lag)
x_val = np.reshape(x_val, (x_val.shape[0],x_val.shape[1],1))
# Create sliding windows into test data
x_test,y_test = create_sliding_windows(array_test_set,len(array_test_set),lag)
x_test = np.reshape(x_test, (x_test.shape[0],x_test.shape[1],1))


In [ ]:
class Double_Tanh(Activation):
    def __init__(self, activation, **kwargs):
        super(Double_Tanh, self).__init__(activation, **kwargs)
        self.__name__ = 'double_tanh'

def double_tanh(x):
    return (tanh(x) * 2)

get_custom_objects().update({'double_tanh':Double_Tanh(double_tanh)})

In [ ]:
from keras.layers import RepeatVector
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
# Hyperparameters
learning_rate = 0.001
hidden_unit = 256
batch_size=3000
epoch = 100

# Architecture Gated Recurrent Unit
regressorGRU = Sequential()

# First GRU layer with dropout
regressorGRU.add(GRU(units=145, return_sequences=True, input_shape=(x_train.shape[1],1)))
regressorGRU.add(Dropout(0.2))
# Second GRU layer with dropout
regressorGRU.add(GRU(units=73, return_sequences=True))
regressorGRU.add(Dropout(0.2))
# Third GRU layer with dropout
regressorGRU.add(GRU(units=73, return_sequences=False))
regressorGRU.add(Dropout(0.2))

# Output layer
regressorGRU.add(Dense(units=289))


# Compiling the Gated Recurrent Unit
regressorGRU.compile(optimizer=optimizers.Adam(learning_rate=learning_rate),loss='mean_squared_error')
print(x_train.shape[1],1)
regressorGRU.summary()
callbacks = [EarlyStopping(patience=15, restore_best_weights=True), ReduceLROnPlateau(factor=0.1, patience=10)]
# Fitting ke data training dan data validation
pred = regressorGRU.fit(x_train, y_train, validation_data=(x_val,y_val), batch_size=batch_size, epochs=epoch, callbacks=callbacks)

In [ ]:
# Graph model loss (train loss & val loss)
fig = plt.figure(figsize=(10, 4))
plt.plot(pred.history['loss'], label='train loss')
plt.plot(pred.history['val_loss'], label='val loss')
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(loc='upper right')
plt.show()

In [ ]:
# Implementation model into data test
y_pred_test = regressorGRU.predict(x_test)

# Invert normalization min-max
y_pred_invert_norm = scaler.inverse_transform(y_pred_test)

In [ ]:
y_pred_invert_norm.shape, x_test.shape

In [ ]:
_y = np.reshape(y_pred_invert_norm, [y_pred_invert_norm.shape[0] * y_pred_invert_norm.shape[1]])
_y.shape

In [ ]:
# Comparison data test with data prediction
datacompare = pd.DataFrame()
datatest=np.array(dataset['Close'][-203328:])
datapred= _y

datacompare['Data Test'] = datatest
datacompare['Prediction Results'] = datapred
# Calculatre value of Root Mean Square Error 
def rmse(datatest, datapred):
    return np.round(np.sqrt(np.mean((datapred - datatest) ** 2)), 4)
print('Result Root Mean Square Error Prediction Model :',rmse(datatest, datapred))

def mape(datatest, datapred): 
    return np.round(np.mean(np.abs((datatest - datapred) / datatest) * 100), 4)
    
print('Result Mean Absolute Percentage Error Prediction Model : ', mape(datatest, datapred), '%')

In [ ]:
# Create graph data test and prediction result
plt.figure(num=None, figsize=(300, 40), dpi=80,facecolor='w', edgecolor='k')
plt.title('Graph Comparison Data Actual and Data Prediction')
plt.plot(datacompare['Data Test'], color='red',label='Data Test')
plt.plot(datacompare['Prediction Results'], color='blue',label='Prediction Results')
plt.xlabel('Day')
plt.ylabel('Price')
plt.legend()
plt.show()